[Table of Contents](../../index.ipynb)

# FRC Analytics with Python - Session 15
# Composition and Inheritance
**Last Updated: 13 May 2021**

In the previous session we learned what a class is. In this session we'll learn a couple techniques for combining classes. *Composition* refers to creating a class with instance properties that are other classes. *Inheritance* refers to having a class acquire features from another class.

## I. Prep Work: Importing Modules from Arbitrary Folders
The examples in this section use the `dice.py` module from the previous session.The following code allows us to import that module into this notebook without having to modify environment variables or other aspects of the system's configuration. The functions used below are not necessary for understanding composition, but they are useful.

In [1]:
import os
import os.path
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(),
                                             os.pardir,
                                             "s14_classes_I")))

from tenthou.dice import Die, Dice, RollableDice

The `sys.path.append(...` statement is pretty dense. The next code cell breaks the line into smaller pieces and explains how it works.

In [2]:
print("Current Working Directory:\t", os.getcwd())
print("Parent Directory:\t\t", os.path.join(os.getcwd(),
                                            os.pardir))
print("Sibling Directory:\t\t", os.path.join(os.getcwd(),
                                             os.pardir,
                                             "s14_classes_I"))
print("Coverted to Absolute path:\t", os.path.abspath(os.path.join(os.getcwd(),
                                                                   os.pardir,
                                                                   "s14_classes_I")))

Current Working Directory:	 C:\Users\stacy\OneDrive\Projects\pyclass_v2\pyclass_frc\sessions\s15_classes_II
Parent Directory:		 C:\Users\stacy\OneDrive\Projects\pyclass_v2\pyclass_frc\sessions\s15_classes_II\..
Sibling Directory:		 C:\Users\stacy\OneDrive\Projects\pyclass_v2\pyclass_frc\sessions\s15_classes_II\..\s14_classes_I
Coverted to Absolute path:	 C:\Users\stacy\OneDrive\Projects\pyclass_v2\pyclass_frc\sessions\s14_classes_I


Here are descriptions of the functions and properties that we used.
* `os.getcwd()`: Gets Python's current working directory. For Jupyter notebooks, the working directory is the directory that contains the notebook.
* `os.pardir`: Returns a string representing a relative path to the parent directory. The value is appropriate for the operating system in use. On most systems the string is two periods: ".."
* `os.path.join()`: Joins a path and folder names to create a longer path. In this example, we join the current working directory, a relative link to the parent directory, and the folder name *s14_classes_I*

## II. Composition
### A. Simple Composition Example
The word *composition* does not refer to any sort of keyword, module, or built-in function. Composition is a principle that is commonly used to structure programs. Consider the following class.

In [ ]:
class DiceSimple():
    def __init__(self, num_dice):
        self.dice = []
        for _ in range(num_dice):
            self.dice.append(Die())
            self.roll()

    def roll(self):
        for die in self.dice:
            die.roll()
            
    def __str__(self):
        return " | ".join([str(die) for die in self.dice])

Dice games would be pretty boring if they used only one die. They usually use several dice. The `DiceSimple` has a `.dice` property that is a list that contains several dice. The next cell creates a `Dice` object with three dice and prints their values.

In [ ]:
# Create a group of three dice
dice = DiceSimple(3)
print(dice)

We can call `.roll()` on the dice object. It calls the `.roll()` method on each individual die.

In [ ]:
# Roll all three dice at once.
dice.roll()
print(dice)

Composition allows us to build sophisticated objects that are relatively simple to understand. The `SimpleDice` object requires only the code needed to join several `Die` objects together. Each separate `Die` object maintains its own value and handles initialization, rolling, and display on its own.

### B. The TenThou Game's `Dice` Class
The code for the `Dice` class below is used in the *ten1000* game. You should be able to understand all of it except for the `@property` statements.

```python
class Dice():
    """A group of dice used to play Ten Thousand.
    
    Attributes:
        __init__(): Takes no arguments.
        add_die(): Adds a die to the group.
        add_dice(): Adds a list of dice to the group.
        ones: The number of die that have value 1.
        fives: The number of die that have value 5.
        triple: Indicates if there are three or more die with
            value 2, 3, 4, or 6.
        score: The score, counting all dice.
        scored_dice: The number of dice that score points.
    """
    def __init__(self):
        """Constructs a new Dice object.
        """
        self.dice = []
        # Tracks how many dice have each value.
        self.counts = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

    def add_die(self, die):
        """Adds a single die to the Dice.
        
        Args:
            die: A tenthou.dice.Die object.
        """
        self.dice.append(die)
        if die.value is not None:
            self.counts[die.value] += 1

    def add_dice(self, dice):
        """Adds a list of dice to the Dice object.
        
        Args:
            dice: A Python list of tenthou.dice.Die objects.
        """
        for die in dice:
            self.add_die(die)

    @property
    def ones(self):
        """The number of dice with value 1."""
        return self.counts[1]

    @property
    def fives(self):
        """The number of dice with value 5."""
        return self.counts[5]

    @property
    def triple(self):
        """Indicates if there are 3 or more die with the same value.
        
        The possible values are 0, 2, 3, 4, and 6. The die with value
        1 or 5 have no impact on this property. Use the ones or fives
        properties to see how many 1 or 5 die are in the group. If 0,
        there are no triples. If 2, then there is are three or more die
        with value 2. If 3, there are three or more die with value 3,
        etc.
        """
        for key, val in self.counts.items():
            if key not in [1, 5]:
                if val >= 3:
                    return key
        return 0
        
    @property
    def score(self):
        """The points available from 1s, 5s, and triples."""
        return (
            self.fives +
            2 * self.ones +
            (14 if self.ones >= 3 else 0) +
            (7 if self.fives >= 3 else 0) +
            2 * self.triple) * 50

    @property
    def scored_dice(self):
        """The number of dice that score points.
        """
        return self.fives + self.ones + (3 if self.triple != 0 else 0)

    def __len__(self):
        return len(self.dice)

    def __str__(self):
        return " | ".join([str(die) for die in self.dice])
```

In [3]:
dice = Dice()
dice.add_dice([Die(), Die(), Die(), Die(), Die()])
print(dice)

None | None | None | None | None


In [ ]:
die = Die()

In [ ]:
str(die)

In [4]:
import inspect

In [5]:
print(inspect.getsource(Dice))

class Dice():
    def __init__(self):
        self.dice = []
        self.counts = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

    def add_die(self, die):
        self.dice.append(die)
        if die.value is not None:
            self.counts[die.value] += 1

    def add_dice(self, dice):
        for die in dice:
            self.add_die(die)

    @property
    def ones(self):
        return self.counts[1]

    @property
    def fives(self):
        return self.counts[5]

    @property
    def triple(self):
        for key, val in self.counts.items():
            if key not in [1, 5]:
                if val >= 3:
                    return key
        return 0
        
    @property
    def score(self):
        return (
            self.fives +
            2 * self.ones +
            (14 if self.ones >= 3 else 0) +
            (7 if self.fives >= 3 else 0) +
            2 * self.triple) * 50

    @property
    def scored_dice(self):
        return self.fives + self.ones + (3 if self.triple !